### Oliveyoung 사이트 크롤링 

#### 1. 상품 목록 조회

In [ ]:

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd
from datetime import datetime

print("라이브러리 준비 완료\n")

# =============================================================================
# 2. 크롤링 설정
# =============================================================================

BASE_URL = "https://www.oliveyoung.co.kr/store/display/getMCategoryList.do"
MAX_PAGES = 5

CATEGORIES = {
    "토너": "100000100010013",
    "에센스/세럼/앰플": "100000100010014",
    "로션": "100000100010016",
}

ALL_PRODUCTS = []

# =============================================================================
# 3. 유틸리티 함수
# =============================================================================

def safe_filename(name):
    """파일명에 사용할 수 없는 문자 제거"""
    # /를 언더스코어로 변경
    return name.replace('/', '_').replace('\\', '_').replace(':', '_')

# =============================================================================
# 4. 드라이버 설정 함수
# =============================================================================

def setup_driver():
    """Selenium 드라이버 초기화"""
    chrome_options = Options()
    
    # Cloudflare 우회
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    
    # 기본 설정
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--window-size=1920,1080')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
    
    # 헤드리스 모드
    chrome_options.add_argument('--headless')
    
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    
    return driver

# =============================================================================
# 5. 제품 정보 추출 함수
# =============================================================================

def extract_product_info(product_element, category_name):
    """제품 기본 정보 + URL 추출"""
    try:
        soup = BeautifulSoup(product_element.get_attribute('outerHTML'), 'html.parser')
        
        # 상세 URL
        link_elem = soup.select_one('a[href*="getGoodsDetail"]')
        if not link_elem or 'href' not in link_elem.attrs:
            return None
        
        detail_url = link_elem['href']
        if not detail_url.startswith('http'):
            detail_url = "https://www.oliveyoung.co.kr" + detail_url
        
        # 브랜드
        brand_elem = soup.select_one('.tx_brand')
        brand = brand_elem.text.strip() if brand_elem else ""
        
        # 제품명
        name_elem = soup.select_one('.tx_name')
        name = name_elem.text.strip() if name_elem else ""
        
        # 현재가
        price_elem = soup.select_one('.prd_price .tx_cur .tx_num')
        price = price_elem.text.strip().replace(',', '') if price_elem else ""
        
        return {
            'category': category_name,
            'brand': brand,
            'name': name,
            'price': price,
            'detail_url': detail_url
        }
        
    except Exception as e:
        return None

# =============================================================================
# 6. 메인 크롤링 실행
# =============================================================================

print("="*70)
print(" 올리브영 제품 목록 크롤링 시작")
print("="*70)

# 드라이버 생성
try:
    driver = setup_driver()
    print(" 브라우저 연결 성공\n")
except Exception as e:
    print(f" 드라이버 초기화 실패: {e}")
    raise

# 크롤링 시작
try:
    for cat_idx, (category_name, disp_cat_no) in enumerate(CATEGORIES.items(), 1):
        print(f"\n{'='*70}")
        print(f"[{cat_idx}/{len(CATEGORIES)}] {category_name}")
        print(f"{'='*70}")
        
        category_products = []
        
        for page_num in range(1, MAX_PAGES + 1):
            url = (
                f"{BASE_URL}?"
                f"dispCatNo={disp_cat_no}&"
                f"prdSort=01&"
                f"rowsPerPage=24&"
                f"pageIdx={page_num}"
            )
            
            try:
                print(f"  📄 {page_num}페이지 로드 중...", end=" ")
                driver.get(url)
                
                # 제품 목록 로드 대기
                WebDriverWait(driver, 30).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "ul.cate_prd_list > li"))
                )
                time.sleep(2)
                
                # 제품 요소 가져오기
                product_elements = driver.find_elements(By.CSS_SELECTOR, "ul.cate_prd_list > li")
                
                if not product_elements:
                    print(" 제품 없음")
                    break
                
                # 제품 정보 추출
                page_count = 0
                for product_elem in product_elements:
                    product_info = extract_product_info(product_elem, category_name)
                    
                    if product_info:
                        # 중복 체크
                        if not any(p['detail_url'] == product_info['detail_url'] for p in category_products):
                            category_products.append(product_info)
                            page_count += 1
                
                print(f"{page_count}개 수집")
                
            except Exception as e:
                print(f"오류: {str(e)[:50]}")
                continue
        
        print(f"\n  {category_name} 카테고리: {len(category_products)}개 수집 완료")
        ALL_PRODUCTS.extend(category_products)
        
        # 카테고리별 중간 저장 (파일명 안전하게!)
        if category_products:
            temp_df = pd.DataFrame(category_products)
            safe_cat_name = safe_filename(category_name)  # 슬래시 제거!
            temp_filename = f"step1_{safe_cat_name}_{datetime.now().strftime('%Y%m%d_%H%M')}.csv"
            temp_df.to_csv(temp_filename, encoding='utf-8-sig', index=False)
            print(f" 저장: {temp_filename}")

finally:
    # 크롤링 완료 후 드라이버 종료
    driver.quit()
    print("\n브라우저 종료")

# =============================================================================
# 7. 결과 확인 및 최종 저장
# =============================================================================

print("\n" + "="*70)
print(f" 크롤링 완료!")
print("="*70)

df_products = pd.DataFrame(ALL_PRODUCTS)

print(f"\n 총 {len(df_products)}개 제품 URL 수집")
print(f"\n카테고리별 분포:")
print(df_products['category'].value_counts())

print("\n샘플 데이터 (상위 5개):")
print(df_products[['category', 'brand', 'name']].head())

# 최종 저장
final_filename = f"oliveyoung_products_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
df_products.to_csv(final_filename, encoding='utf-8-sig', index=False)

print(f"\n최종 저장: {final_filename}")
print(f"다음 단계: 리뷰 크롤링 준비 완료!")
print("="*70)

✅ 라이브러리 준비 완료

🚀 올리브영 제품 목록 크롤링 시작
✅ 브라우저 연결 성공


📁 [1/3] 토너
  📄 1페이지 로드 중... ✅ 24개 수집
  📄 2페이지 로드 중... ✅ 24개 수집
  📄 3페이지 로드 중... ✅ 24개 수집
  📄 4페이지 로드 중... ✅ 24개 수집
  📄 5페이지 로드 중... ✅ 24개 수집

  🎯 토너 카테고리: 120개 수집 완료
  💾 저장: step1_토너_20251110_1506.csv

📁 [2/3] 에센스/세럼/앰플
  📄 1페이지 로드 중... ✅ 24개 수집
  📄 2페이지 로드 중... ✅ 24개 수집
  📄 3페이지 로드 중... ✅ 24개 수집
  📄 4페이지 로드 중... ✅ 24개 수집
  📄 5페이지 로드 중... ✅ 24개 수집

  🎯 에센스/세럼/앰플 카테고리: 120개 수집 완료
  💾 저장: step1_에센스_세럼_앰플_20251110_1506.csv

📁 [3/3] 로션
  📄 1페이지 로드 중... ✅ 24개 수집
  📄 2페이지 로드 중... ✅ 24개 수집
  📄 3페이지 로드 중... ✅ 24개 수집
  📄 4페이지 로드 중... ✅ 24개 수집
  📄 5페이지 로드 중... ✅ 24개 수집

  🎯 로션 카테고리: 120개 수집 완료
  💾 저장: step1_로션_20251110_1507.csv

✅ 브라우저 종료

🎉 크롤링 완료!

📊 총 360개 제품 URL 수집

카테고리별 분포:
category
토너           120
에센스/세럼/앰플    120
로션           120
Name: count, dtype: int64

🔍 샘플 데이터 (상위 5개):
  category  brand                                               name
0       토너    라네즈  [미스트 기획/화잘먹] 라네즈 크림스킨 170ml 리필기획 (+170ml 리필+50...
1       토너  바이오더마    [11월 올영

#### 2. 개별 리뷰 분석

In [1]:
# ============================================================================
# 올리브영 리뷰 크롤링 - 최종 통합 코드 (undetected-chromedriver)
# ============================================================================

# 1. 라이브러리 설치 및 임포트
import sys
try:
    import undetected_chromedriver as uc
except:
    print("undetected-chromedriver 설치 중...")
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "undetected-chromedriver"])
    import undetected_chromedriver as uc

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd
from datetime import datetime

print("라이브러리 준비 완료")

# ============================================================================
# 2. 드라이버 설정
# ============================================================================

def setup_driver():
    options = uc.ChromeOptions()
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--window-size=1920,1080')
    
    driver = uc.Chrome(options=options)
    return driver

# ============================================================================
# 3. 리뷰 파싱 함수
# ============================================================================

def parse_single_review(review_elem):
    """단일 리뷰 요소에서 데이터 추출"""
    try:
        # 리뷰어 닉네임
        reviewer = review_elem.select_one('.user .id')
        reviewer_name = reviewer.text.strip() if reviewer else ""
        
        # 평점
        rating_elem = review_elem.select_one('.review_point .point')
        if rating_elem and 'style' in rating_elem.attrs:
            width = rating_elem['style'].replace('width:', '').replace('%', '').strip()
            try:
                rating = float(width) / 20
            except:
                rating = 0
        else:
            rating = 0
        
        # 날짜
        date_elem = review_elem.select_one('.score_area .date')
        review_date = date_elem.text.strip() if date_elem else ""
        
        # 리뷰 내용
        content_elem = review_elem.select_one('.txt_inner')
        content = content_elem.text.strip() if content_elem else ""
        
        # 피부타입, 피부고민, 자극도
        poll_items = review_elem.select('.poll_sample dl')
        poll_data = {}
        for item in poll_items:
            dt = item.select_one('dt span')
            dd = item.select_one('dd span.txt')
            if dt and dd:
                poll_data[dt.text.strip()] = dd.text.strip()
        
        # 도움이 돼요 수
        helpful_elem = review_elem.select_one('.recom_area .num')
        helpful_count = helpful_elem.text.strip() if helpful_elem else "0"
        
        return {
            'reviewer': reviewer_name,
            'rating': rating,
            'date': review_date,
            'content': content,
            'skin_type': poll_data.get('피부타입', ''),
            'skin_concern': poll_data.get('피부고민', ''),
            'irritation': poll_data.get('자극도', ''),
            'helpful_count': helpful_count
        }
        
    except Exception as e:
        return None

# ============================================================================
# 4. 리뷰 추출 함수
# ============================================================================

def extract_reviews(driver, detail_url,max_reviews=80):
    """단일 제품의 모든 리뷰를 추출"""
    reviews = []
    
    try:
        # 제품 상세 페이지 로드
        driver.get(detail_url)
        time.sleep(5)
        
        # 페이지 스크롤
        driver.execute_script("window.scrollTo(0, 1500);")
        time.sleep(2)
        
        # 리뷰 버튼 클릭
        try:
            review_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "a.goods_reputation[data-attr*='리뷰']"))
            )
            driver.execute_script("arguments[0].click();", review_button)
            time.sleep(3)
        except:
            print("  리뷰 버튼 없음")
            return []
        
        # 리뷰 영역으로 스크롤 (Ajax 트리거)
        driver.execute_script("window.scrollTo(0, 2000);")
        time.sleep(5)
        
        driver.execute_script("window.scrollBy(0, 300);")
        time.sleep(5)
        
        # 리뷰 목록 로드 대기
        try:
            WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "ul#gdasList"))
            )
            time.sleep(2)
        except:
            print("  리뷰 로드 실패")
            return []
        
        # 페이지 순회
        page_num = 1
        while True:
            print(f"    페이지 {page_num} 수집 중...", end=" ")
            
            # 현재 페이지 HTML 파싱
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            
            # 리뷰 목록
            review_elements = soup.select('ul#gdasList > li')
            
            if not review_elements:
                print("리뷰 없음")
                break
            
            page_reviews = 0
            for review_elem in review_elements:
                if len(reviews) >= max_reviews:  # <- 여기 추가
                        return reviews
                    
                review_data = parse_single_review(review_elem)
                if review_data:
                    reviews.append(review_data)
                    page_reviews += 1
            
            print(f"{page_reviews}개")
            
            # 다음 페이지로 이동
            try:
                # 현재 페이지 번호
                current_page = driver.find_element(By.CSS_SELECTOR, ".pageing strong")
                current_num = int(current_page.text)
                
                # 다음 페이지 링크
                next_page_link = driver.find_element(
                    By.CSS_SELECTOR, 
                    f".pageing a[data-page-no='{current_num + 1}']"
                )
                
                # 클릭
                driver.execute_script("arguments[0].click();", next_page_link)
                time.sleep(3)
                
                page_num += 1
                
            except:
                # 다음 페이지 없으면 종료
                break
        
    except Exception as e:
        print(f"  오류: {str(e)[:50]}")
    
    return reviews

# ============================================================================
# 5. 메인 크롤링 실행
# ============================================================================

print("\n" + "="*70)
print("올리브영 리뷰 크롤링 시작")
print("="*70)

# 제품 목록 로드
try:
    df_products = pd.read_csv("oliveyoung_products.csv")
    print(f"\n제품 목록 로드 완료: {len(df_products)}개 제품")
except Exception as e:
    print(f"오류: CSV 파일을 찾을 수 없습니다 - {e}")
    raise

all_reviews = []

# 이미 수집한 제품 확인
collected_urls = set()
try:
    existing_df = pd.read_csv("reviews_checkpoint_latest.csv")
    collected_urls = set(existing_df['product_url'].unique())
    all_reviews = existing_df.to_dict('records')
    print(f"기존 데이터 로드: {len(collected_urls)}개 제품 이미 완료 (누적 {len(all_reviews)}개 리뷰)")
except:
    print("새로 시작")

# 드라이버 초기화
driver = setup_driver()
print("브라우저 연결 완료\n")

try:
    for idx, row in df_products.iterrows():
        product_url = row['detail_url']
        
        # 이미 수집한 제품 건너뛰기
        if product_url in collected_urls:
            print(f"[{idx+1}/{len(df_products)}] 건너뜀 (이미 수집)")
            continue
        
        product_name = row['name']
        brand = row['brand']
        category = row['category']
        
        print(f"\n[{idx+1}/{len(df_products)}] {brand} - {product_name}")
        
        # 리뷰 추출
        reviews = extract_reviews(driver, product_url,max_reviews=100)
        
        # 제품 정보 추가
        for review in reviews:
            review['product_name'] = product_name
            review['product_url'] = product_url
            review['brand'] = brand
            review['category'] = category
        
        all_reviews.extend(reviews)
        
        print(f"  완료: 총 {len(reviews)}개 리뷰")
        
        # 매 5개 제품마다 중간 저장
        if (idx + 1) % 5 == 0:
            temp_df = pd.DataFrame(all_reviews)
            temp_df.to_csv("reviews_checkpoint_latest.csv", encoding='utf-8-sig', index=False)
            print(f"  중간 저장 완료 (누적 {len(all_reviews)}개 리뷰)")
        
        time.sleep(2)
        
finally:
    driver.quit()
    print("\n브라우저 종료")

# ============================================================================
# 6. 최종 결과 저장
# ============================================================================

print("\n" + "="*70)
print("크롤링 완료")
print("="*70)

if all_reviews:
    final_df = pd.DataFrame(all_reviews)
    final_filename = f"oliveyoung_reviews_final_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    final_df.to_csv(final_filename, encoding='utf-8-sig', index=False)
    
    print(f"\n총 수집 리뷰: {len(all_reviews)}개")
    print(f"최종 저장: {final_filename}")
    
    print(f"\n카테고리별 리뷰 수:")
    print(final_df['category'].value_counts())
    
    print(f"\n평점 분포:")
    print(final_df['rating'].value_counts().sort_index())
else:
    print("\n수집된 리뷰가 없습니다.")

print("="*70)

라이브러리 준비 완료

올리브영 리뷰 크롤링 시작

제품 목록 로드 완료: 360개 제품
새로 시작
브라우저 연결 완료


[1/360] 라네즈 - [미스트 기획/화잘먹] 라네즈 크림스킨 170ml 리필기획 (+170ml 리필+50ml+미스트펌프)
    페이지 1 수집 중... 10개
    페이지 2 수집 중... 10개
    페이지 3 수집 중... 10개
    페이지 4 수집 중... 10개
    페이지 5 수집 중... 10개
    페이지 6 수집 중... 10개
    페이지 7 수집 중... 10개
    페이지 8 수집 중... 10개
    페이지 9 수집 중... 10개
    페이지 10 수집 중... 10개
    페이지 11 수집 중...   완료: 총 100개 리뷰

[2/360] 바이오더마 - [11월 올영픽] 바이오더마 하이드라비오 토너 500ml 기획(+화장솜 20매 증정)
    페이지 1 수집 중... 10개
    페이지 2 수집 중... 10개
    페이지 3 수집 중... 10개
    페이지 4 수집 중... 10개
    페이지 5 수집 중... 10개
    페이지 6 수집 중... 10개
    페이지 7 수집 중... 10개
    페이지 8 수집 중... 10개
    페이지 9 수집 중... 10개
    페이지 10 수집 중... 10개
    페이지 11 수집 중...   완료: 총 100개 리뷰

[3/360] 토리든 - [증량기획] 토리든 다이브인 저분자 히알루론산 토너 300ml 기획(+100ml 추가 증정)
    페이지 1 수집 중... 10개
    페이지 2 수집 중... 10개
    페이지 3 수집 중... 10개
    페이지 4 수집 중... 10개
    페이지 5 수집 중... 10개
    페이지 6 수집 중... 10개
    페이지 7 수집 중... 10개
    페이지 8 수집 중... 10개
    페이지 9 수집 중... 10개
    페이지 10 수집 중... 

#### 3. 데이터 품질 확인

In [10]:
!pip install undetected-chromedriver

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for undetected-chromedriver: filename=undetected_chromedriver-3.5.5-py3-none-any.whl size=47131 sha256=122ba7cc973e483cd90fc13da3e33d9e73dd0bc333ec1d70bbb942347e2c93c9
  Stored in directory: c:\users\황동민\appdata\local\pip\cache\wheels\7a\5f\c1\06f68421cc7172ef51504631252870bcb3a2fdf3b6a025f362
Successfully built undetected-chromedriver
